### Mental Health Facilities in California – Analysis

#### By Irene Casado Sánchez

Mental Health Care Datasets:
   * Licensed and Certified Healthcare Facility Listing – Source: https://data.chhs.ca.gov/dataset/healthcare-facility-locations
   * Licensed Mental Health Rehabilitation Centers (MHRC) and Psychiatric Health Facilities (PHF) – Source: https://data.chhs.ca.gov/dataset/licensed-mental-health-rehabilitation-centers-mhrc-and-psychiatric-health-facilities-phf
   * Licensed Narcotic Treatment Programs – Source: https://data.chhs.ca.gov/dataset/licensed-narcotic-treatment-programs
   * SUD Recovery Treatment Facilities – Source: https://data.chhs.ca.gov/dataset/sud-recovery-treatment-facilities

Death Profiles by County – Source: https://www.cdph.ca.gov/Programs/CHSI/Pages/County-Health-Status-Profiles.aspx
Statewide Death Profiles – Source: https://data.chhs.ca.gov/dataset/statewide-death-profiles
    
Main Questions: 

* Which counties have the most mental health facilities? (total numbers and per population)
* Which counties have the fewest mental health facilities? (total numbers and per population)
* What is the percentage of deaths related to mental health problems in each county? 

Main fields used in this analysis:

* `county`
* `capacity`
* `population`
* `deaths`


#### Data Limitations:

* This analysis is based on the aforementioned databases, therefore, potentially private health centers do not appear in this analysis. 

#### Key Findings:

* According to our analysis, California accounts 2132 mental health care facilities. More than a half of them (1192) are located in 3 counties: Los Angeles, Orange and San Diego.
* Rural counties as Alpine, Del Norte, Lassen, Madera, Modoc, Mono and Plumas have 0 facilities.
* The suicide rate in these areas rural areas is particularly high. 
* California records an average of 4,226 suicide deaths annually.

_____

In [1]:
import os
from pathlib import Path

import pandas as pd
import pandas_bokeh
import altair as alt
import numpy as np
pandas_bokeh.output_notebook()
pd.set_option('plotting.backend', 'pandas_bokeh')

# Create Bokeh-Table with DataFrame:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, output_file

from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.models import LogColorMapper
from bokeh.palettes import Viridis6 as palette
from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.unemployment import data as unemployment


pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.0f}'.format

Loading BokehJS ...

Let's read the data:

In [2]:
DATA_DIR = os.environ['DATA_DIR']

In [54]:
facilities = pd.read_csv(DATA_DIR + "/processed/all-facilities-csv.csv") 
mental_deaths = pd.read_csv(DATA_DIR + "/processed/california_mental_health_deaths.csv")
ca_fips = pd.read_csv(DATA_DIR + "/processed/ca_fips.csv", dtype={'county': object})
ca_population = pd.read_csv(DATA_DIR + "/processed/ca_population.csv",dtype={'county': object})
total_deaths_suicide = pd.read_csv(DATA_DIR + "/processed/suicide_deaths_ca.csv")

### Data vetting and merging:

In [55]:
ca_fips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  58 non-null     int64 
 1   fips        58 non-null     int64 
 2   name        58 non-null     object
 3   state       58 non-null     object
 4   fip_clean   58 non-null     int64 
 5   county      58 non-null     object
dtypes: int64(3), object(3)
memory usage: 2.8+ KB


In [56]:
ca_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  58 non-null     int64 
 1   P1_001N     58 non-null     int64 
 2   state       58 non-null     int64 
 3   county      58 non-null     object
dtypes: int64(3), object(1)
memory usage: 1.9+ KB


Now, we are going to merge ca_fips with ca_population.

In [57]:
merge_fips_pop = pd.merge(ca_population, ca_fips, on="county", how="outer")
merge_fips_pop_clean = merge_fips_pop[['P1_001N', 'name']]

In [58]:
merge_fips_pop_clean['county'] = merge_fips_pop_clean.name.str.replace(' County', '').str.upper() # Create a new column with the name of the county.

/var/folders/5t/ptdbrrps2zzc0q0jvwkbsh980000gn/T/ipykernel_50384/805703852.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_fips_pop_clean['county'] = merge_fips_pop_clean.name.str.replace(' County', '').str.upper() # Create a new column with the name of the county.


In [59]:
merge_fips_pop_clean.head(3)

,P1_001N,name,county
0,1204,Alpine County,ALPINE
1,45292,Calaveras County,CALAVERAS
2,27743,Del Norte County,DEL NORTE


In [60]:
mental_deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   county                         58 non-null     object
 1   suicide_2015_2017              58 non-null     object
 2   suicide_2018_2020              58 non-null     object
 3   drug_induced_deaths_2015_2017  58 non-null     object
 4   drug_induced_deaths_2018_2020  58 non-null     object
dtypes: object(5)
memory usage: 2.4+ KB


In [61]:
mental_deaths.head(5) # /* Rates with a relative standard error greater than or equal to 23 percent are deemed unreliable.

,county,suicide_2015_2017,suicide_2018_2020,drug_induced_deaths_2015_2017,drug_induced_deaths_2018_2020
0,ALAMEDA,8.6,8.8,9.9,15.4
1,ALPINE,-,58.3 *,-,116.6 *
2,AMADOR,21.2,33.7,16.0 *,25.4
3,BUTTE,18.9,22.1,27.8,31.4
4,CALAVERAS,26.1,24.3,18.0,25.5


In [62]:
facilities.info() # Let's check the different columns.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2132 entries, 0 to 2131
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Column          2132 non-null   int64  
 1   facility_name   2132 non-null   object 
 2   facility_type   2132 non-null   object 
 3   facility_clean  2132 non-null   object 
 4   address         2132 non-null   object 
 5   city            2132 non-null   object 
 6   county          2132 non-null   object 
 7   state           2132 non-null   object 
 8   zip             2132 non-null   object 
 9   capacity        2132 non-null   int64  
 10  latitude        2132 non-null   float64
 11  longitude       2132 non-null   float64
 12  phone_number    2122 non-null   object 
 13  other_details   1811 non-null   object 
 14  full_address    2132 non-null   object 
dtypes: float64(2), int64(2), object(11)
memory usage: 250.0+ KB


In [63]:
facilities.county.unique()

array(['ALAMEDA', 'AMADOR', 'BUTTE', 'CONTRA COSTA', 'FRESNO', 'HUMBOLDT',
       'IMPERIAL', 'KERN', 'LOS ANGELES', 'MARIN', 'MENDOCINO',
       'MONTEREY', 'ORANGE', 'EL DORADO', 'PLACER', 'RIVERSIDE', 'MERCED',
       'SACRAMENTO', 'SAN BERNARDINO', 'SAN DIEGO', 'SAN FRANCISCO',
       'SAN JOAQUIN', 'SAN LUIS OBISPO', 'SAN MATEO', 'SANTA BARBARA',
       'SANTA CLARA', 'SANTA CRUZ', 'SOLANO', 'SONOMA', 'STANISLAUS',
       'SHASTA', 'YUBA', 'NEVADA', 'TULARE', 'VENTURA', 'YOLO', 'NAPA',
       'CALAVERAS', 'COLUSA', 'GLENN', 'INYO', 'KINGS', 'LAKE',
       'MARIPOSA', 'SAN BENITO', 'SIERRA', 'SISKIYOU', 'SUTTER',
       'TRINITY', 'TEHAMA', 'TUOLUMNE'], dtype=object)

Now, we are going to create a new dataframe grouping our data by county and number of facilities.

In [64]:
facilities_per_county = facilities.groupby(["county"]).full_address.count().reset_index() # Filter the data using the required columns

In [65]:
facilities_per_county.rename(columns = {"full_address":"facility_count"}, inplace = True) # Rename the column to avoid confusion

In [66]:
facilities_per_county.sort_values(by='facility_count', ascending=False).head(10) # Here, we are checking which counties have more mental health care facilities.

,county,facility_count
15,LOS ANGELES,626
23,ORANGE,392
29,SAN DIEGO,174
25,RIVERSIDE,120
28,SAN BERNARDINO,64
26,SACRAMENTO,64
30,SAN FRANCISCO,56
35,SANTA CLARA,49
48,VENTURA,48
0,ALAMEDA,48


In [67]:
facilities_per_county.sort_values(by='facility_count', ascending=True).head(10) # Here, we are checking which counties have more mental health care facilities.

,county,facility_count
1,AMADOR,1
4,COLUSA,1
38,SIERRA,1
8,GLENN,1
47,TUOLUMNE,1
17,MARIPOSA,1
45,TRINITY,1
44,TEHAMA,2
11,INYO,2
39,SISKIYOU,3


Now, we are going to merge our two databases (facilities + deaths related to mental health per county):

In [68]:
county_facilities_death = pd.merge(facilities_per_county, mental_deaths, on='county', how='outer')

Next, we are goin to merge the data with the merge_fips_pop_clean:

In [69]:
full_data = pd.merge(county_facilities_death, merge_fips_pop_clean, on='county', how='outer')

In [70]:
full_data.rename(columns = {'P1_001N': 'population', 'name': 'full_county_name'}, inplace = True) # Renaming the columns for more clarity.
#full_data.to_csv(DATA_DIR + '/processed/full_data.csv')
#full_data.to_json(DATA_DIR + '/processed/full_data.json')

In [71]:
total_deaths_suicide_2020 = total_deaths_suicide[total_deaths_suicide.year == 2020] # As we need the total deaths on 2020, we have to filter this dataset

In [72]:
total_deaths_suicide_2020['county'] = total_deaths_suicide_2020.county.str.upper()

/var/folders/5t/ptdbrrps2zzc0q0jvwkbsh980000gn/T/ipykernel_50384/3670409774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_deaths_suicide_2020['county'] = total_deaths_suicide_2020.county.str.upper()


In [73]:
full_data = pd.merge(total_deaths_suicide_2020, full_data, on='county') # Merging new dataframe

In [74]:
full_data.rename(columns = {'year_x': 'year', 'deaths_x': 'number_of_deaths', 'cause_y': 'cause'}, inplace = True)

In [75]:
full_data.head(5)

,year,county,geography_type,cause,deaths,facility_count,suicide_2015_2017,suicide_2018_2020,drug_induced_deaths_2015_2017,drug_induced_deaths_2018_2020,population,full_county_name
0,2020,ALAMEDA,Residence,Intentional self-harm (suicide),141,48,8.6,8.8,9.9,15.4,1682353,Alameda County
1,2020,ALPINE,Residence,Intentional self-harm (suicide),0,NaN,-,58.3 *,-,116.6 *,1204,Alpine County
2,2020,AMADOR,Residence,Intentional self-harm (suicide),12,1,21.2,33.7,16.0 *,25.4,40474,Amador County
3,2020,BUTTE,Residence,Intentional self-harm (suicide),43,12,18.9,22.1,27.8,31.4,211632,Butte County
4,2020,CALAVERAS,Residence,Intentional self-harm (suicide),13,4,26.1,24.3,18.0,25.5,45292,Calaveras County


## Analysis: 

In [92]:
full_data.head(3)

,year,county,geography_type,cause,deaths,facility_count,suicide_2015_2017,suicide_2018_2020,drug_induced_deaths_2015_2017,drug_induced_deaths_2018_2020,population,full_county_name,facilities_rate,suicide_2018_2020_clean,drug_induced_deaths_2018_2020_clean
0,2020,ALAMEDA,Residence,Intentional self-harm (suicide),141,48,8.6,8.8,9.9,15.4,1682353,Alameda County,0,9,15
1,2020,ALPINE,Residence,Intentional self-harm (suicide),0,NaN,-,58.3 *,-,116.6 *,1204,Alpine County,NaN,58,117
2,2020,AMADOR,Residence,Intentional self-harm (suicide),12,1,21.2,33.7,16.0 *,25.4,40474,Amador County,0,34,25


Let's create some graphs to find the counties with more mental health care facilities:

In [77]:
source = full_data

chart_totals=alt.Chart(source).mark_bar().encode(
    x=alt.X('facility_count'),
    y=alt.Y('county', sort='-x'),
    tooltip=['facility_count'],
).properties(width=450, title='Facilities per County in CA'
)

chart_totals

alt.Chart(...)

In [78]:
source = full_data

chart_top10=alt.Chart(source).mark_bar().encode(
    x=alt.X('facility_count'),
    y=alt.Y('county', sort='-x'),
    tooltip=['facility_count'],
).properties(width=450, title='Top 10 Counties with More Mental Health Care Facilities'
).transform_window(
    rank='rank(facility_count)',
    sort=[alt.SortField('facility_count', order='descending')]
).transform_filter(
    (alt.datum.rank < 10)
).configure_bar(
    opacity=0.8,
    color='orange'
)

chart_top10

alt.Chart(...)

In [79]:
source = full_data

chart_top10less=alt.Chart(source).mark_bar().encode(
    x=alt.X('facility_count'),
    y=alt.Y('county', sort='-x'),
    tooltip=['facility_count'],
).properties(width=450, title='Top 10 Counties with Less Mental Health Care Facilities'
).transform_window(
    rank='rank(facility_count)',
    sort=[alt.SortField('facility_count', order='ascending')]
).transform_filter(
    (alt.datum.rank < 10)
).configure_bar(
    opacity=0.8,
    color='orange'
)

chart_top10less

alt.Chart(...)

Let's create some graphs matching the number of establishments with the population. 

But first, we need to calculate the rate of facilities per 1000 residents in each county.

In [80]:
full_data['facilities_rate'] = ((full_data['facility_count'])/(full_data['population']/10000)).round(2) # Number of facilities per 1000 people?

In [81]:
source = full_data

chart_top10pop=alt.Chart(source).mark_bar().encode(
    x=alt.X('facilities_rate'),
    y=alt.Y('county', sort='-x'),
    tooltip=['facilities_rate'],
).properties(width=450, title='Top 10 Counties with More Mental Health Care Facilities Per Capita (10,000)'
).transform_window(
    rank='rank(facilities_rate)',
    sort=[alt.SortField('facilities_rate', order='descending')]
).transform_filter(
    (alt.datum.rank < 10)
).configure_bar(
    opacity=0.8,
    color='orange'
)

chart_top10pop

alt.Chart(...)

In [82]:
source = full_data

chart_top10popless=alt.Chart(source).mark_bar().encode(
    x=alt.X('facilities_rate'),
    y=alt.Y('county', sort='-x'),
    tooltip=['facilities_rate'],
).properties(width=450, title='Top 10 Counties with Less Mental Health Care Facilities Per Capita (10,000)'
).transform_window(
    rank='rank(facilities_rate)',
    sort=[alt.SortField('facilities_rate', order='ascending')]
).transform_filter(
    (alt.datum.rank < 20)
).configure_bar(
    opacity=0.8,
    color='orange'
)

chart_top10popless

alt.Chart(...)

Now, let's check the counties with higher rates of deaths related to mental health issues.

In [83]:
full_data['suicide_2018_2020_clean'] = full_data['suicide_2018_2020'].str.replace('*', '').replace("\xa0", "")
full_data['drug_induced_deaths_2018_2020_clean'] = full_data['drug_induced_deaths_2018_2020'].str.replace('*', '').replace("\xa0", "")
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 57
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   year                                 58 non-null     int64  
 1   county                               58 non-null     object 
 2   geography_type                       58 non-null     object 
 3   cause                                58 non-null     object 
 4   deaths                               47 non-null     float64
 5   facility_count                       51 non-null     float64
 6   suicide_2015_2017                    58 non-null     object 
 7   suicide_2018_2020                    58 non-null     object 
 8   drug_induced_deaths_2015_2017        58 non-null     object 
 9   drug_induced_deaths_2018_2020        58 non-null     object 
 10  population                           58 non-null     int64  
 11  full_county_name                  

/var/folders/5t/ptdbrrps2zzc0q0jvwkbsh980000gn/T/ipykernel_50384/842759591.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  full_data['suicide_2018_2020_clean'] = full_data['suicide_2018_2020'].str.replace('*', '').replace("\xa0", "")
/var/folders/5t/ptdbrrps2zzc0q0jvwkbsh980000gn/T/ipykernel_50384/842759591.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  full_data['drug_induced_deaths_2018_2020_clean'] = full_data['drug_induced_deaths_2018_2020'].str.replace('*', '').replace("\xa0", "")


In order to make our calculations, we need to change the type of our columns, from object to float.

In [84]:
full_data['suicide_2018_2020_clean'] = full_data['suicide_2018_2020_clean'].astype(float)
full_data['drug_induced_deaths_2018_2020_clean'] = full_data['drug_induced_deaths_2018_2020_clean'].astype(float)
full_data.info() # Checking the changes (the decimals are not there...?)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 57
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   year                                 58 non-null     int64  
 1   county                               58 non-null     object 
 2   geography_type                       58 non-null     object 
 3   cause                                58 non-null     object 
 4   deaths                               47 non-null     float64
 5   facility_count                       51 non-null     float64
 6   suicide_2015_2017                    58 non-null     object 
 7   suicide_2018_2020                    58 non-null     object 
 8   drug_induced_deaths_2015_2017        58 non-null     object 
 9   drug_induced_deaths_2018_2020        58 non-null     object 
 10  population                           58 non-null     int64  
 11  full_county_name                  

In [87]:
suicide_data = full_data[['county', 'facility_count','population', 'deaths']]

In [88]:
suicide_data['deaths_rate'] = ((suicide_data['deaths'])/(suicide_data['population']/20000)).round(2) # Number of facilities per 1000 people?

/var/folders/5t/ptdbrrps2zzc0q0jvwkbsh980000gn/T/ipykernel_50384/678326785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suicide_data['deaths_rate'] = ((suicide_data['deaths'])/(suicide_data['population']/20000)).round(2) # Number of facilities per 1000 people?


In [90]:
# Let's create a scater plot with drug induced deaths

source = suicide_data

alt.Chart(source).mark_circle(size=60).encode(
    y='population:Q',
    x='deaths:Q',
    tooltip=['county', 'facility_count', 'population', 'deaths'],
).properties(width=450, title='2020 Deaths by Suicide per County in CA'
).configure_mark(
    opacity=0.4,
    color='orange'
).interactive()

alt.Chart(...)

In [91]:
source = suicide_data

alt.Chart(source).mark_circle(size=60).encode(
    y='population:Q',
    x='deaths_rate:Q',
    tooltip=['county', 'facility_count', 'population', 'deaths', 'deaths_rate'],
).properties(width=450, title='2020 Deaths by Suicide per County in CA per Capita (20,000)'
).configure_mark(
    opacity=0.4,
    color='orange'
).interactive()

alt.Chart(...)

Now, let make a graph with the suicide deaths over time in CA.

In [94]:
total_deaths_suicide_year = total_deaths_suicide.groupby(by= ["year"])['deaths'].sum().reset_index() # Group by year and sum the total deaths

In [95]:
total_deaths_suicide_year

,year,deaths
0,2014,"4,116"
1,2015,"4,113"
2,2016,"4,224"
3,2017,"4,227"
4,2018,"4,422"
5,2019,"4,387"
6,2020,"4,092"


In [96]:
mean_suicide = total_deaths_suicide_year[["deaths"]].mean()
print(mean_suicide)

deaths   4,226
dtype: float64


In [293]:
source = total_deaths_suicide_year

bar = alt.Chart(source).mark_bar().encode(
    x='year:O',
    y='deaths:Q'
)

line = alt.Chart(source).mark_line(color='red').transform_window(
    # The field to average
    rolling_mean='mean(deaths)',
    # The number of values before and after the current value to include.
    frame=[-9, 0]
).encode(
    x='year:O',
    y='deaths:Q'
)

(bar + line).properties(width=350, title='Deaths by Suicide Over Time in CA'
).configure_mark(
    opacity=0.4,
    color='orange'
)

alt.LayerChart(...)